In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_898a030403f7485eb967ba42b0aa73e9 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_898a030403f7485eb967ba42b0aa73e9 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_898a030403f7485eb967ba42b0aa73e9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pBEYHEt6kq6wZUv4qf0CrCa-RwJvFHgMgC81kXLzcrIe',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_898a030403f7485eb967ba42b0aa73e9)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_898a030403f7485eb967ba42b0aa73e9.get_object(Bucket='rklb-donotdelete-pr-rdv6sie7airyfr', Key='Iris.csv (2).ipynb')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 


In [3]:
import tensorflow.compat.v1 as tf 
tf.compat.v1.disable_eager_execution()

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4594386863221931056
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6829857122635831826
physical_device_desc: "device: XLA_CPU device"
]


In [5]:
#품종 column을 one-hot-encode
iris_data_one_hot_encoded = pd.get_dummies(df_data_1)
iris_data_one_hot_encoded.head(5)

NameError: name 'df_data_1' is not defined

## Visualize the Data

In [ ]:
import matplotlib.pyplot as plt

names = {'Iris-setosa','Iris-versicolor','Iris-virginica'}
feature_names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
labels=df_data_1.iloc[:,5]

In [ ]:
# Visualize the data sets
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
for target, target_name in enumerate(names):
    X_plot = df_data_1[labels== target_name]
    plt.plot(X_plot.iloc[:, 1], X_plot.iloc[:, 2], linestyle='none', marker='o', label=target_name)
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.axis('equal')
plt.legend();


plt.subplot(1, 2, 2)
for target, target_name in enumerate(names):
    X_plot = df_data_1[labels== target_name]
    plt.plot(X_plot.iloc[:, 3], X_plot.iloc[:, 4], linestyle='none', marker='o', label=target_name)
plt.xlabel(feature_names[2])
plt.ylabel(feature_names[3])
plt.axis('equal')
plt.legend();

In [ ]:
#전체 데이터를 80%은 트레이닝 20% 테스트로 쪼갬
iris_train_data = iris_data_one_hot_encoded.sample(frac=0.8, random_state=200)
iris_test_data = iris_data_one_hot_encoded.drop(iris_train_data.index)

In [ ]:
#input은 꽃잎의 너비와길이, 꽃받침의 너비와길이
#output은 세개중 하나로
iris_train_input_data = iris_train_data.filter(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'])
iris_train_label_data = iris_train_data.filter(['Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica'])
iris_test_input_data = iris_test_data.filter(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'])
iris_test_label_data = iris_test_data.filter(['Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica'])


In [ ]:
#x는 input값을 위한 placeholder
#w는 가중치
#b는 편차
#y는 트레이닝해서 나온 결과(가설)
#y_는 진짜 결과값
x = tf.placeholder(tf.float32,[None, 4])
W = tf.Variable(tf.zeros([4, 3]))
b = tf.Variable(tf.zeros([3]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 3])

In [ ]:
#cross_entropy를 cost함수로
cross_entropy  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=tf.matmul(x,W)+b))

In [ ]:
#cost를 최소화
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
#30,000번 학습
#tf.global_variables_initializer().run()
epoch_history = []
loss_history = []

sess.run(tf.global_variables_initializer())
for _ in range(30000):
    #Usually send batches to the training step. But since the dataset is small sending all
    l,a=sess.run([cross_entropy,train_step], feed_dict={x: iris_train_input_data, y_: iris_train_label_data})
    epoch_history.append(_)
    loss_history.append(l)

print("trained")

## Crossentropy vs. epoch

In [ ]:
#Draw Cross Entropy Graph
plt.xlabel('Epochs')
plt.ylabel('crossentropy')


# Show the cross_entropy
plt.plot(epoch_history, loss_history)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
#정확도
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy : ', sess.run(accuracy, feed_dict={x: iris_test_input_data, y_: iris_test_label_data}))

In [ ]:
import numpy as np

In [ ]:
#추론
predict_x = np.array([[6.9,3.1,5.4,2.1]])
prediction=tf.argmax(y,1)
print('예측값:', sess.run(prediction, feed_dict={x: predict_x}))

In [ ]:
predict_x = np.array([[5.1,3.3,1.7,0.5]])
prediction=tf.argmax(y,1)
print('예측값:', sess.run(prediction, feed_dict={x: predict_x}))

## ROC Curve

In [ ]:
# quality of the neural net using ROC curve and AUC
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

#test인풋의 예측값
test_prob=sess.run(tf.nn.softmax(logits=tf.matmul(x,W)+b), feed_dict={x: iris_test_input_data})
#print(test_prob.ravel())

#test인풋의 label
test_label=sess.run(tf.argmax(y_,1), feed_dict={y_: iris_test_label_data})
#print(test_label)

#test인풋의 binarized
bi_test_label = label_binarize(test_label, classes=[0, 1, 2])
#print(bi_test_label)


In [ ]:
# send the actual dependent variable classifications for param 1, 
# and the confidences of the true classification for param 2.
#roc_curve는 multiclass&multi_Indicater의 입력이 되지 않으므로 ravel()을 통해 1차원배열로 펼침
fpr,tpr,thr= roc_curve(bi_test_label.ravel(), test_prob.ravel())
AUC = auc(fpr,tpr)
plt.plot(fpr, tpr,label='%s(area = %0.2f)' %('Model',AUC))

plt.legend(loc="lower right")
#plt.xlabel('위양성률(Fall-Out)')
#plt.ylabel('재현률(Recall)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

## Confusion Matrix

In [ ]:
#confusion_matrix
from sklearn.metrics import confusion_matrix

print('label : ',test_label)
test_pred=sess.run(tf.argmax(y,1), feed_dict={x: iris_test_input_data})
print('predict: ',test_pred)

confM=confusion_matrix(test_label,test_pred)
print(confM)